In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download data

In [2]:
#"hubdistance_rutas_ejesviales" se hizo con puntos a cada 20 metros --> [Funciona bien con un mincount de 9 puntos, pero necesita algo de limpieza después en GIS]
#"hubdistance_rutas20m_ejesviales10m" se hizo con rutas a cada 20 metros y ejesviales a cada 10 metros

In [3]:
ejesviales = gpd.read_file('../../data/external/temporal_todocker/marcogeo_ejesviales_2km.gpkg')
hubdistance = gpd.read_file('../../data/external/temporal_todocker/hubdistance_rutas10m_ejesviales10m.gpkg')

In [4]:
print(ejesviales.shape)
ejesviales.head(1)

(19887, 14)


,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,rutas_transporte,geometry
0,140390001,14,039,0001,03975,00007,Patria,Avenida,Dos sentidos,2,Urbana,1403900010397500007,NaN,"MULTILINESTRING ((668243.519 2290955.957, 6682..."


In [5]:
print(hubdistance.shape)
hubdistance.head(2)

(1381164, 14)


,cat,Tipo_de_se,Clasificac,Clasifica0,idruta,fid_2,cat_2,Tipo_de_se_2,Clasificac_2,Clasifica0_2,idruta_2,HubName,HubDist,geometry
0,1,Complementaria,Rutas complementarias y otros servicios,Operando,1,2,1,Complementaria,Rutas complementarias y otros servicios,Operando,1,1403900010366300013,1.883139,POINT (671655.896 2288719.052)
1,1,Complementaria,Rutas complementarias y otros servicios,Operando,1,1155,2,Complementaria,Rutas complementarias y otros servicios,Operando,2,1403900010366300013,11.513562,POINT (671655.450 2288709.068)


In [6]:
columns_tokeep = ['idruta','HubName','geometry']
hubdistance_f = hubdistance[columns_tokeep]
hubdistance_f = hubdistance_f.rename(columns={'HubName':'segmentocalle'})
hubdistance_f.head(1)

,idruta,segmentocalle,geometry
0,1,1403900010366300013,POINT (671655.896 2288719.052)


In [7]:
rutas = list(hubdistance_f.idruta.unique())
print(len(rutas))

segmentos = list(hubdistance_f.segmentocalle.unique())
print(len(segmentos))

176
3735


## Iteración (Método anterior, no permite ajustar el número de veces que se debe encontrar una ruta en un segmento para considerar que pasa por ahí y no solo es una intersección)

In [89]:
#hubdistance_f.head(2)

In [90]:
# ---------- Diccionario de segmentos que tienen rutas y las rutas que tienen
#dicc_segmentos_rutas = {}
#listavacia = []

# ---------- Crear un diccionario con los segmentos de calle y una lista de rutas por segmento
#for index,row in hubdistance_f.iterrows():    
#    segmento_calle = row['segmentocalle'] #Es un str   
#    ruta = row['idruta'] #Es un int
    
    # ---------- Si es la primera vez que encuentra este segmento, agregarlo con una lista vacía
#    if segmento_calle not in dicc_segmentos_rutas:
#        dicc_segmentos_rutas[segmento_calle] = listavacia.copy()

    # ---------- Para esta row en la que iteramos (segmento de calle y ruta), si es la primera vez que encuentra la ruta en este segmento, agregarla a esa lista:
#    if ruta not in dicc_segmentos_rutas[segmento_calle]:
#        dicc_segmentos_rutas[segmento_calle].append(ruta)

## Data processing 01 - Diccionario con segmentos y rutas que pasan por cada segmento

In [8]:
#Rutas en el gdf
rutas = list(hubdistance_f.idruta.unique())

In [9]:
#---------- Insumos base de la iteración ----------
dicc_segmentos_rutas = {} #Guarda los segmentos de calle y las rutas que pasan por cada segmento de calle
listavacia = [] #Lista vacía usada cuando se encuentra un nuevo segmento de calle

#---------- Para decidir si es una intersección o una calle muy pequeña ----------

# Con rutas @10m sin borrar duplicados y segmentos @10m sin duplicados (Iteración actual): 
# Cuando una ruta pasa por un segmento de calle corto (Cuadra pequeña, aprox 55 metros), se encuentran aprox. 10 puntos, pero en calles más pequeñas (50 mts) hay aprox. 8 puntos.
# Cuando una ruta pasa por una intersección 

mincount = 9 #Número mínimo de ocasiones en las que se debe encontrar una ruta cerca de un segmento de calle para considerar que la ruta pasa por ahí y no solo que la encontró porque es una intersección
    
#---------- Iterar ruta por ruta ----------
for ruta in rutas:
    gdf_ruta = hubdistance_f[hubdistance_f.idruta == ruta]

    #---------- Veces que la ruta pasa por un segmento de calle ----------
    segs_deruta = gdf_ruta.groupby('segmentocalle').count()
    #---------- Formato ----------
    columns_tokeep = ['idruta']
    segs_deruta = segs_deruta[columns_tokeep]
    segs_deruta = segs_deruta.reset_index()
    segs_deruta = segs_deruta.rename(columns={'idruta':'count'})

    #---------- Para el count de segmentos esa ruta ----------
    for row_id,row_series in segs_deruta.iterrows():
        segmento_calle = row_series[0]
        count = row_series[1]
    
        #---------- Si se confirma que la ruta pasa por ahí, agregar al diccionario el segmento de calle y la ruta ----------
        if count > (mincount-1): 
            #---------- Si no se había encontrado ese segmento de calle, meterlo al diccionario por primera vez con una lista vacia de rutas ----------
            if segmento_calle not in dicc_segmentos_rutas:
                dicc_segmentos_rutas[segmento_calle] = listavacia.copy()
            
            #---------- Si la ruta no está ya asignada a ese segmento, meterla a la lista ----------
            if ruta not in dicc_segmentos_rutas[segmento_calle]:
                dicc_segmentos_rutas[segmento_calle].append(ruta)

In [16]:
#Ejemplo:
dicc_segmentos_rutas['1403900010341300007']

[152]

## Data processing 02 - Asignar numero de rutas a los segmentos de ejesviales

In [11]:
# Crear un gdf de ejes viales con rutas_count
ejesviales_rutas = ejesviales.copy()
ejesviales_rutas['rutas_count'] = 0

In [12]:
ejesviales_rutas.head(1)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,rutas_transporte,geometry,rutas_count
0,140390001,14,039,0001,03975,00007,Patria,Avenida,Dos sentidos,2,Urbana,1403900010397500007,NaN,"MULTILINESTRING ((668243.519 2290955.957, 6682...",0


In [13]:
# Iterar en el diccionario creado para asignar el numero de rutas que pasan por cada segmento
for segmento in dicc_segmentos_rutas:
    idx = ejesviales_rutas['CVEGEOSEG'] == segmento
    ejesviales_rutas.loc[idx,'rutas_count'] = len(dicc_segmentos_rutas[segmento])

In [14]:
# Revisión rápida: En esta df (prueba) máximo deberían haber 14 rutas en una calle
ejesviales_rutas = ejesviales_rutas.sort_values(by = 'rutas_count', ascending = False)
ejesviales_rutas.head(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,rutas_transporte,geometry,rutas_count
18461,141200001,14,120,0001,00302,00002,López Mateos Sur,Avenida,Dos sentidos,2,Urbana,1412000010030200002,NaN,"MULTILINESTRING ((663924.928 2280302.722, 6639...",37
14446,141200001,14,120,0001,03541,00003,López Mateos Sur,Avenida,Dos sentidos,2,Urbana,1412000010354100003,NaN,"MULTILINESTRING ((666046.183 2283779.958, 6660...",35


## Save to db

In [15]:
ejesviales_rutas = ejesviales_rutas.to_crs("EPSG:4326")
aup.gdf_to_db_slow(ejesviales_rutas,"rutastransporte_bystreet", 'segundopiso', if_exists='replace')